### 1º Voy a priorizar en primer lugar a los  diseñadores y desarrolladores (lugares con aglomeración de empresas de videojuegos y tecnológicas que hayan facturado más de 1M en su historia, las de videojuegos cumplen el perfil en este caso)

Me quedo de la base de datos Companies con las empresas de videojuegos que han facturado más de 1M, ejecuto el script que escribe el json con la consulta filtrando la base de datos todo con el formato adecuado

In [1]:
from getJsonGames1M import writeJsonOffice
writeJsonOffice()

Hago el mongoimport e importo la colección a la base de datos,indexo el campo position como índice geoespacial. Ahora puedo buscar los lugares donde se aglomeran estas empresas, y buscar la lista de ciudades. El scrip devuelve una lista de ciudades donde hay empresas que a 100km a la redonda tienen más de 27 empresas de videojuegos que han facturado más de 1M, creo que es suficiente para satisfacer a los desarrolladores y a los diseñadores. 

In [2]:
from getCitiesDevNDesigner import citiesDevNDesignerOK
citiesDevNDes = citiesDevNDesignerOK()

In [3]:
print(citiesDevNDes[1])

{'city': 'Brooklyn', 'lat': 40.714952833333335, 'long': -73.9684315}


### 2º En las anteriores ciudades voy a buscar qué tal son las comunicaciones, así cumplo con el requisito de los ejecutivos, para ello ejecuto el script que buscará aeropuertos, estaciones de tren y autobús con la api de google place pasándole el diccionario de las ciudades con sus coordenadas. El criterio va a ser que al menos haya 1 aeropuerto y 10 estaciones de tren o metro en 20 km a la redonda. Es de suponer que todas las ciudades que cumplan el criterio también tendrán buenas comunicaciones por carretera.

In [4]:
from filterCitiesForExecutive import getCities

In [5]:
cities = getCities(citiesDevNDes)

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.3) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [6]:
len(cities)

38

### 3º Ahora voy a buscar ciudades con una buena aglomeración de Starbucks, pienso que 10 en 10 km a la redonda es adecuado

In [9]:
from filterCitiesStarbucks import filterStarbucks

In [10]:
cities = list(filter(filterStarbucks,cities))

### 4º El siguiente filtro que voy a aplicar a mi lista es uno que satisfaga a las familias, me ha parecido que el criterio de que haya más de 5 lugares con la palabra universidad en 10 km cuadrados es un buen criterio, si hay universidades también hay colegios y guarderías para los niños

In [11]:
from filterCitiesSchool import filterUniversity
cities = list(filter(filterUniversity,cities))


In [12]:
len(cities)

31

In [17]:
"""import pandas as pd
pd.DataFrame(cities).to_csv('../outputs/citiesStepUni.csv')"""

### 5º Ahora voy a buscar que haya una buenna aglomeración de bares en la ciudad, si hay bares habrá discotecas y demás establecimientos de ocio nocturno y diurno. 100 bares en 10 km

In [13]:
from filterCitiesBar import filterBar
c = list(filter(filterBar,cities))

In [14]:
len(c)

30